##  Demo timeline extraction
This is a demo to use chatGPT to extract the timeline of events from Woo-decision letters. To try the demo out, make sure there is 1 pdf file in the demo/input folder. Then, run the below cel and the output will be a sorted dataframe with the dates with an event, their class and event phrases. Depending on the length of document it will take about a 30 second to 3 minutes.

In [3]:
from code.scripts.filter import filter_and_extract
from code.scripts.date_correction import uncorrected_dates
from code.scripts.date_correction import compile
from code.scripts.decision import decision_class
from code.scripts.isodates import convert_date

from dotenv import load_dotenv, find_dotenv

import shutil
import glob
import os
import pickle
import openai
import pandas as pd


def run_demo():
    # delete output for each run
    folder_path = 'demo/output'

    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)

    print('Checking if pdf is text or image based...')
    filter_and_extract("demo/input", "demo/output/txt", 0.1)


    # check quality of pdf & extract text
    folder_path = 'demo/output/txt'
    file_extension = '*.txt'

    txt_files = glob.glob(f"{folder_path}/{file_extension}")
    if len(txt_files) == 0:
        print("ERROR: pdf quality is too low -> pdf is image-based. Select different pdf as input.")
        return None
    else:
        print("Completed: Text extracted from pdf")

    # extract dates from text
    print(f"Selecting dates and sentences...")
    uncorrected_dates("demo/output/txt/", "demo/output", "")
    print("Completed: dates extracted from text.")

    
    # correct dates
    with open('demo/output/uncorrected_dates_.pkl', 'rb') as fp:
        test_uncorrected = pickle.load(fp)

    print(f"Correcting dates...")
    # get second results of test set
    test_corrected = compile(test_uncorrected)
    print(f"Completed: dates corrected.")


    # extracting the decision date
    print(f"Extracting the decision date...")
    test_decision = decision_class(test_corrected)
    test_decision['id'] = range(len(test_decision))

    timeline = test_decision.copy()
    timeline.loc[timeline['decisiondate'] == True, 'decisiondate'] = "besluit datum"
    timeline = timeline.rename(columns={"decisiondate":"class"})
    timeline = timeline.loc[timeline['class'] != False]
    print(f"Completed: decision date extracted")

    # set up chatGPT
    print(f"Setting up API of ChatGPT...")
    _ = load_dotenv(find_dotenv()) # read local .env file

    # store api in text file, extract here
    with open("code/api.txt", 'r') as f:
        api = f.read()

    # openai.api_key  = os.getenv(api)
    openai.api_key = api
    print(f"Completed: API of ChatGPT is set")

    print(f"Loading training data...")
    gt_train = pd.read_csv("code/data/GT/GTtrain/date_event_combinations.csv")
    # gt_train['class'] = gt_train['label'].apply(lambda x: x.split(': ')[1] if ': ' in x else x)
    print(f"Completed: training data loaded")

    from code.scripts.chatgpt_extraction import run
    print(f"Classifying dates into True/False & extracting event phrase...")
    run(test_decision.loc[test_decision['decisiondate'] == False], 1, "demo/output/", gt_train)
    print(f"Completed: Dates classified into with and without event & event phrases extracted")

    # select dates with an event
    predictions = pd.read_csv("demo/output/predictions.csv")
    predictions = predictions.loc[predictions['prediction_class'] == True]  

    print(f"Classifying dates with an event into an event class...")
    input_chatgpt = predictions.drop(columns=['prediction_class', 'batch', 'batch_runtime', 'run_id', 'prompt'])

    from code.scripts.chatgpt_classification import run
    # gt = pd.read_csv("data/GTtrain1-6/date_event_combinations.csv")
    try_run = run(input_chatgpt, 1, "demo/output/chatgpt", gt_train)
    print(f"Completed: dates with an event classified into event classes")

    # dropping unnessecarry columns
    prediction_class = pd.read_csv("demo/output/chatgptpredictions.csv")
    prediction_class_clean = prediction_class.drop(columns=['decisiondate', 'batch', 'batch_runtime', 'run_id', 'prompt'])
    prediction_class_clean = prediction_class_clean.rename(columns={'prediction':'class'})
    timeline = pd.concat([prediction_class_clean, timeline])

    # convert dates in ISO_date
    print("Converting dates into ISO date format & sort dates...")
    timeline = convert_date(timeline).sort_values(by=['ISO_date'])
    show_timeline = timeline.drop(columns=['label', 'corrected', 'start_in_text', 'end_in_text', 'start_in_sent', 'end_in_sent', 'id', 'complete_date', 'doc_id'])
    show_timeline = show_timeline[['date', 'class', 'prediction_event', 'sentence', 'ISO_date']]
    print("Completed: dates converted to ISO date and sorted")

    # show complete sentences
    pd.set_option('display.max_colwidth', None)
    display(show_timeline)
    
    return show_timeline
 




In [4]:
timeline = run_demo()

Checking if pdf is text or image based...
('text', 'Besluit+op+Wob-verzoek+over+informatie+over+diertellingen+Bredesteeg+37+te+Echteld.pdf')
Completed: Text extracted from pdf
Selecting dates and sentences...
at document 1 out of 1
Completed: dates extracted from text.
Correcting dates...
Completed: dates corrected.
Extracting the decision date...
Completed: decision date extracted
Setting up API of ChatGPT...
Completed: API of ChatGPT is set
Loading training data...
Completed: training data loaded
Classifying dates into True/False & extracting event phrase...
Completed: Dates classified into with and without event & event phrases extracted
Classifying dates with an event into an event class...
Completed: dates with an event classified into event classes
Converting dates into ISO date format & sort dates...
Completed: dates converted to ISO date and sorted


,date,class,prediction_event,sentence,ISO_date
0,6 oktober 2021,verzoek datum,u heeft met een beroep op de Wet openbaarheid van bestuur verzocht om informatie over diertellingen van het bedrijf Bredesteeg 37 te Echteld,", In uw e-mail van 6 oktober 2021 heeft u met een beroep op de Wet openbaarheid van bestuur (hierna: Wob) verzocht om informatie over diertellingen van het bedrijf Bredesteeg 37 te Echteld.",2021-10-06
1,13 oktober 2021,ontvangst verzoek bevestigd,De ontvangst van uw verzoek is schriftelijk bevestigd,Procedure De ontvangst van uw verzoek is schriftelijk bevestigd bij brief van 13 oktober 2021 met kenmerk WOB/2021/252.,2021-10-13
2,3 februari 2022,beslistermijn verdaagd,de beslistermijn met twee weken is verlengd vanwege het vragen van zienswijzen aan derden,In de brief van 3 februari 2022 is aan u medegedeeld dat de beslistermijn met twee weken is verlengd vanwege het vragen van zienswijzen aan derden.,2022-02-03
3,11 maart 2022,overig,Verzoek openbaarmaking gegevens,"Rijksdienst voor Ondernemend Nederland Vergunningen en Handhaving Datum 11 maart 2022 Onze referentie WOB/2021/252 Relevante artikelen uit de Wob Bijlage nummer 1 Horend bij Verzoek openbaarmaking gegevens Datum Onze referentie WOB/2021/252 Contactpersoon Relevante artikelen uit de Wob Artikel 1 In deze wet en de daarop berustende bepalingen wordt verstaan onder: a. document: een bij een bestuursorgaan berustend schriftelijk stuk of ander materiaal dat gegevens bevat; b. bestuurlijke aangelegenheid: een aangelegenheid die betrekking heeft op beleid van een bestuursorgaan, daaronder begrepen de voorbereiding en de uitvoering ervan; c. intern beraad: het beraad over een bestuurlijke aangelegenheid binnen een bestuursorgaan, dan wel binnen een kring van bestuursorganen in het kader van de gezamenlijke verantwoordelijkheid voor een bestuurlijke aangelegenheid; d. niet-ambtelijke adviescommissie: een van overheidswege ingestelde instantie, met als taak het adviseren van een of meer bestuursorganen en waarvan geen ambtenaren lid zijn, die het bestuursorgaan waaronder zij ressorteren adviseren over de onderwerpen die aan de instantie zijn voorgelegd.",2022-03-11
4,11 maart 2022,overig,2.,11 maart 2022 2.,2022-03-11
5,11 maart 2022,verzoek datum,"Het bestuursorgaan verstrekt de informatie met betrekking tot de documenten die de verlangde informatie bevatten door: Datum a. kopie ervan te geven of de letterlijke inhoud ervan in andere vorm te 11 maart 2022 verstrekken, Onze referentie b. kennisneming van de inhoud toe te staan, WOB/2021/252 c. een uittreksel of een samenvatting van de inhoud te geven, of d. inlichtingen daaruit te verschaffen.","Het bestuursorgaan verstrekt de informatie met betrekking tot de documenten die de verlangde informatie bevatten door: Datum a. kopie ervan te geven of de letterlijke inhoud ervan in andere vorm te 11 maart 2022 verstrekken, Onze referentie b. kennisneming van de inhoud toe te staan, WOB/2021/252 c. een uittreksel of een samenvatting van de inhoud te geven, of d. inlichtingen daaruit te verschaffen.",2022-03-11
6,11 maart 2022,overig,"afwijking van het eerste lid, aanhef en onder c, het verstrekken van milieu- 11 maart 2022 informatie uitsluitend achterwege blijft voorzover het belang van openbaarmaking niet Onze referentie opweegt tegen het daar genoemde belang.","Voorts blijft in Datum afwijking van het eerste lid, aanhef en onder c, het verstrekken van milieu- 11 maart 2022 informatie uitsluitend achterwege voorzover het belang van openbaarmaking niet Onze referentie opweegt tegen het daar genoemde belang.",2022-03-11
7,11 maart 2022,verzoek ontvangen,Inventarislijst opgevraagde gegevens,Rijksdienst voor Ondernemend Nederland Vergunningen en Handhaving Inventarislijst opgevraagde gegevens Datum 11 maart 2022 Bijlage nummer 2 Onze referentie Horend bij Verzoek openbaarmaking gegevens WOB/2021/252 Datum Onze referentie WOB/2021/252 Contactpersoon Inventarislijst opgevraagde gegevens Nummer Document WOB Art.,2022-03

In [5]:
# markdown_table = timeline.to_markdown()
# with open('README.md', 'w') as file:
#     file.write(markdown_table)